# RNNを使った文書分類（解答例）

In [429]:
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import datasets

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 訓練データから語彙集合を作成
* データの読み込みや前処理は`torchtext`の旧バージョン（0.9より前）とかなり変わっているので注意。

In [430]:
import torchtext
torchtext.__version__

'0.10.1'

* データセットの訓練データ部分をもとに語彙セットを作成する。
 * 語彙セットを作成するとき、テストデータ部分を使わないように。

In [431]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
#tokenizer = get_tokenizer('spacy') # やや時間がかかる。

# IMDbデータの訓練データ部分のイテレータを取得
train_iter = IMDB(split='train', root='/home/masada/.data/')

# 各文書をtokenizeするヘルパ関数
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

# 最小の出現頻度をmin_freqで指定して語彙サイズを調整する
vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  specials=["<unk>", "<pad>"],
                                  min_freq=10)

# OoVトークンには"<unk>"のインデックスを返すことにする
vocab.set_default_index(vocab["<unk>"]) 

In [432]:
print(len(vocab))

20437


In [433]:
vocab(['here', 'is', 'an', 'example'])

[132, 10, 41, 465]

In [434]:
vocab(['<unk>', '<pad>'])

[0, 1]

### IMDBデータを分割してデータセットを作成

In [435]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter, test_iter = IMDB(root='/home/masada/.data/')
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# 訓練データ部分から検証データを切り出す
num_train = int(len(train_dataset) * 0.9)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

### RNNへの入力として使えるミニバッチを作る関数

In [436]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
  labels = {'neg':0, 'pos':1}
  label_list, text_list = [], []
  for _label, _text in batch:
    label_list.append(labels[_label])
    processed_text = torch.tensor(vocab(tokenizer(_text)), dtype=torch.int64)
    text_list.append(processed_text)
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = pad_sequence(text_list, batch_first=False, padding_value=PAD_IDX)
  return label_list.to(device), text_list.to(device)

In [437]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=collate_batch)

In [438]:
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)

## モデルの定義
* LSTMを使う（GRUに変えても良い）
 * http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [439]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx,
               num_layers=1, bidirectional=False, p=0.5):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.dropout = p

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers)
    self.fc1 = nn.Linear(hid_dim * (1 + num_layers), hid_dim)
    self.fc2 = nn.Linear(hid_dim, num_class)
    self.af = nn.ReLU()
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # srcの形は[単語列長, バッチサイズ]

    embedded = self.dropout(self.embedding(src))
    # embeddedの形は[単語列長, バッチサイズ, 埋め込み次元数]

    outputs, (hidden, _) = self.rnn(embedded)
    # outputsの形は[単語列長, バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[1, バッチサイズ, 隠れ状態の次元数]

    mean_outputs = outputs.mean(0)
    hidden = hidden.transpose(0, 1)
    hidden = hidden.reshape(hidden.shape[0], -1)
    # mean_outputsの形は[バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[バッチサイズ, 隠れ状態の次元数]
    output = self.fc2(self.af(self.fc1(torch.cat((mean_outputs, hidden), dim=1))))

    return output

In [440]:
def train(model, loader, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0.
  epoch_acc = 0.
  for label, text in loader:
    output = model(text)
    loss = criterion(output, label)
    loss.backward()

    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item()
    epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

In [441]:
def evaluate(model, loader, criterion):
  model.eval()

  epoch_loss = 0.
  epoch_acc = 0.
  with torch.no_grad():
    for label, text in loader:
      output = model(text)
      loss = criterion(output, label)
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

In [442]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time // 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [443]:
INPUT_DIM = len(vocab)
NUM_CLASS = 2
EMBED_DIM = 128
HIDDEN_DIM = 128
NUM_LAYERS = 2

In [444]:
model = RNNTextSentiment(EMBED_DIM, HIDDEN_DIM, NUM_CLASS, INPUT_DIM,
                         num_layers=NUM_LAYERS, bidirectional=True,
                         padding_idx=PAD_IDX, p=0.5).to(device)

In [445]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,929,666 trainable parameters


In [446]:
optimizer = optim.Adam(model.parameters(), lr=0.0003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [447]:
criterion = nn.CrossEntropyLoss()

In [448]:
N_EPOCHS = 50
CLIP = 1.

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, CLIP)
  valid_loss, valid_acc = evaluate(model, valid_dataloader, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  scheduler.step()

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc * 100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc * 100:.1f}% (valid)')

Epoch 1 | time in 0 minutes, 16 seconds
	Loss 0.6921 (train)	|	Acc 50.8% (train)
	Loss 0.6761 (valid)	|	Acc 57.3% (valid)
Epoch 2 | time in 0 minutes, 16 seconds
	Loss 0.6486 (train)	|	Acc 62.7% (train)
	Loss 0.6009 (valid)	|	Acc 69.0% (valid)
Epoch 3 | time in 0 minutes, 16 seconds
	Loss 0.5914 (train)	|	Acc 68.8% (train)
	Loss 0.5861 (valid)	|	Acc 70.1% (valid)
Epoch 4 | time in 0 minutes, 16 seconds
	Loss 0.5148 (train)	|	Acc 75.2% (train)
	Loss 0.5115 (valid)	|	Acc 77.6% (valid)
Epoch 5 | time in 0 minutes, 16 seconds
	Loss 0.4635 (train)	|	Acc 78.7% (train)
	Loss 0.5419 (valid)	|	Acc 76.4% (valid)
Epoch 6 | time in 0 minutes, 16 seconds
	Loss 0.4336 (train)	|	Acc 80.0% (train)
	Loss 0.4023 (valid)	|	Acc 82.4% (valid)
Epoch 7 | time in 0 minutes, 16 seconds
	Loss 0.4018 (train)	|	Acc 82.4% (train)
	Loss 0.3853 (valid)	|	Acc 84.4% (valid)
Epoch 8 | time in 0 minutes, 15 seconds
	Loss 0.3709 (train)	|	Acc 83.8% (train)
	Loss 0.3461 (valid)	|	Acc 86.6% (valid)
Epoch 9 | time in 0 minu

## テストデータで評価

In [449]:
loss, acc = evaluate(model, test_dataloader, criterion)
print(f'\tLoss {loss:.4f} (test)\t|\tAcc {acc * 100:.1f}% (test)')

	Loss 0.2765 (test)	|	Acc 90.3% (test)
